# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902014


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:22,  2.84s/it]

Rendering models:  10%|▉         | 3/31 [00:05<00:58,  2.08s/it]

Rendering models:  13%|█▎        | 4/31 [00:06<00:42,  1.57s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:44,  1.70s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:15<00:01,  2.83it/s]

Rendering models:  94%|█████████▎| 29/31 [00:15<00:00,  2.35it/s]

Rendering models:  97%|█████████▋| 30/31 [00:15<00:00,  2.48it/s]

Rendering models: 100%|██████████| 31/31 [00:16<00:00,  2.67it/s]

equidad1                              0.045222
CTidwell3                             0.000324
Audburrito                            0.000355
xperry9702                            0.103676
kayleebug2017                         0.002131
ElisabethB                            0.000192
not-logged-in-ea8ed61a7274cfb6a43b    7.709534
not-logged-in-dacb6c2f9eb6080c5e9f    0.002559
Thedunkmasta                          0.003147
not-logged-in-6d073ad4cbbd00d3ccf3    0.022664
not-logged-in-84be6415a703f6e6a7e1    0.000393
not-logged-in-409241128adb6e9c5c86    0.010154
not-logged-in-22877e3a7af4aafb750a    0.002491
not-logged-in-7e969a324309f376dde8    0.001177
not-logged-in-314d48142a3a122ba666    0.002192
crush202020                           0.000217
pangeli5                              0.000168
SDamian                               0.000173
megamikey0903                         0.099032
Lavadude                              0.003283
not-logged-in-4ea23f5fffc8b98635af    0.001568
Tessaje04    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())